# Overview
This script aims to pridict list of mitre attack technique given a techinque to detect. This will help to give the use a means to know which techniques run following a script that was choses. Eg. User picks T1027 and runs the sript. if the technique is detected in the log, then he is recommended with a list of other techniques to run a search on. 

The algoritm follows a statistical approach where it finds the most number of times the technique were used togather. The dataset used for this statistical analysis is done by using APT actors from MITRE Attack repository. Once we get the common pairs, we form a weighted graph and then using nearest neighbour finder to find the list of techniques that is related to each other.

In [80]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pprint import pprint
from collections import Counter

from pyattck import Attck

import networkx as nx

In [81]:
# Init pyatt lib
attck = Attck()

In [82]:
# Gather and store edge pairs
sourcelist = []
targetlist = []

for actor in attck.enterprise.actors:
    edge_i = [x.id for x in actor.techniques]
    edge_j = [x.id for x in actor.techniques]
    
    for technique_i in edge_i:
        for technique_j in edge_j:
            if technique_j != technique_i:
                sourcelist.append(technique_i)
                targetlist.append(technique_j)

In [83]:
# User couner to counter number of time edges appear to create weights
counter = Counter(tuple(sorted(tup)) for tup in zip(sourcelist,targetlist))
counter

Counter({('T1027', 'T1566.001'): 68,
         ('T1053.005', 'T1566.001'): 44,
         ('T1204.002', 'T1566.001'): 100,
         ('T1566.001', 'T1571'): 16,
         ('T1036.004', 'T1566.001'): 12,
         ('T1059.005', 'T1566.001'): 50,
         ('T1105', 'T1566.001'): 64,
         ('T1027', 'T1053.005'): 46,
         ('T1027', 'T1204.002'): 68,
         ('T1027', 'T1571'): 18,
         ('T1027', 'T1036.004'): 14,
         ('T1027', 'T1059.005'): 40,
         ('T1027', 'T1105'): 66,
         ('T1053.005', 'T1204.002'): 42,
         ('T1053.005', 'T1571'): 12,
         ('T1036.004', 'T1053.005'): 12,
         ('T1053.005', 'T1059.005'): 24,
         ('T1053.005', 'T1105'): 44,
         ('T1204.002', 'T1571'): 16,
         ('T1036.004', 'T1204.002'): 14,
         ('T1059.005', 'T1204.002'): 48,
         ('T1105', 'T1204.002'): 62,
         ('T1036.004', 'T1571'): 8,
         ('T1059.005', 'T1571'): 16,
         ('T1105', 'T1571'): 18,
         ('T1036.004', 'T1059.005'): 8,
         ('

In [84]:
# Store each value in array list
sourcelist = []
targetlist = []
weightlist = []
for key, value in counter.items():
    sourcelist.append(key[0])
    targetlist.append(key[1])
    weightlist.append(value)

In [85]:
# check if the len are same value
print(len(sourcelist))
print(len(targetlist))
print(len(weightlist))

13930
13930
13930


In [86]:
# Store the nodes and weights in dataframe
edges = pd.DataFrame(
    {
        "source": sourcelist,#sourceId,
        "target": targetlist, #targetlist
        "weight": weightlist
    }
)

In [87]:
# Create a graph using the dataframe
G = nx.from_pandas_edgelist(edges, edge_attr=True)

In [104]:
# Run prediction on techniques
technic_neighbours = G.neighbors('T1542.002')
technic_neighbours = [x for x in technic_neighbours]
technic_neighbours

['T1109', 'T1480.001', 'T1120', 'T1564.005']

In [105]:
for item in technic_neighbours:
    for technique in attck.enterprise.techniques:
        if item == technique.id:
            print(item, technique.name, technique.tactics)
        for subtech in technique.subtechniques:
            if item == subtech.id:
                print(item, technique.name, technique.tactics)

T1480.001 Execution Guardrails [AttckTactic('Defense Evasion', 'TA0005')]
T1120 Peripheral Device Discovery [AttckTactic('Discovery', 'TA0007')]
T1564.005 Hide Artifacts [AttckTactic('Defense Evasion', 'TA0005')]
